In [ ]:
# The Scenario: The "Titanic" Survival Predictor 🚢 You have a list of passengers. Some survived, some didn't. Your job is to build a model that predicts survival based on their data.

# The Dataset:

# Python
# import seaborn as sns
# # Load the raw dataset
# df = sns.load_dataset('titanic')

# # Inspect it carefully - there are missing values and categorical strings!
# print(df.head())
# print(df.isnull().sum())
# Tasks:

# Clean:

# Drop columns that seem useless (e.g., deck, embark_town, alive which is a duplicate of target).

# Handle missing values in age (maybe fill with the mean/median?).

# Drop rows where embarked is missing (it's only 2 rows).

# Encode: Convert categorical columns (sex, embarked) into numbers (Get Dummies or manual mapping).

# Split: Train/Test Split (80/20).

# Model: Train an XGBoost Classifier.

# Evaluate: Print the Accuracy and a Confusion Matrix.

# Constraints:

# You must fix the errors (missing values) before the model will run. XGBoost can handle some missing data, but sklearn metrics often complain if you aren't careful.

# Try to get an accuracy above 80%.

In [4]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, ConfusionMatrixDisplay
# from sklearn.model_selection import XGBClassifier

df = sns.load_dataset('titanic')

In [ ]:
df.to_csv("C:\\Users\\Sumed\\Desktop\\Project\titanic.csv")

In [ ]:
# exploratory data analysis - understanding how the data looks like.

print("initial data look: ")
display(df.isnull().sum())
display(df.describe())
display(df.shape) #(891, 15)
# i can see there are 177 rows where age is null/missing, column "deck" has 688 null values, embarked as 2 and embark_town also has 2. I've decided to drop the nulls in embarked and embark_town


df_filtered = df[df['embarked'].notnull() | df['embark_town'].notnull()]
print('-----------------------data look post embarked and embarked_town filter: ')
display(df_filtered.isnull().sum())
display(df_filtered.shape) #(889, 15)
#so there is a total loss of 2 rows meaning the nulls were in the same two rows for both embarked and embark_town columns, i need to handle age and deck columns as they have a large number of NANs/Nulls.

df_filtered_2 = df_filtered.drop(columns = ['deck'])
print('-----------------------data look post deck column drop: ')
display(df_filtered_2.isnull().sum())
display(df_filtered_2.shape) #(889, 14)

df_filtered_3 = df_filtered_2.copy()
df_filtered_3['age'] = df_filtered_2['age'].fillna(df_filtered_2['age'].median())
print('-----------------------data look post mediam filled in age column: ')
display(df_filtered_3.isnull().sum())
display(df_filtered_3.shape) #(889, 14)
#now i can further process the dataframe into meaningful data that the models can understand. There are some columns which are not logically contributing to the survival of the person onboard the ship. 


df_filtered_4 = df_filtered_3.drop(columns = [


    alive
embark_town
embarked
fare
class
])


initial data look: 


survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


(891, 15)

-----------------------data look post embarked and embarked_town filter: 


...

(889, 14)

-----------------------data look post mediam filled in age column: 


survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

(889, 14)

KeyError: "['deck'] not found in axis"

In [12]:
display(df_filtered_3[['who','sex']].drop_duplicates())

,who,sex
0,man,male
1,woman,female
7,child,male
9,child,female


In [ ]:
display(df_filtered_3)





,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,Southampton,yes,True
888,0,3,female,28.0,1,2,23.4500,S,Third,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,Cherbourg,yes,True


In [3]:
display(df)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True
